## 1. 패키지 로드

In [1]:
import pandas as pd
import numpy as np
import pickle as pk

from gensim.models import FastText
from sqlalchemy import create_engine
from tqdm import tqdm_notebook
from nltk import sent_tokenize

## 2. 판례 데이터 로드

### 1) DB 정보

In [2]:
host = 'localhost'
user = 'root'
password = '1234'
db = 'cases'
port = 3306

### 2) DB 연동

In [3]:
db_connection_str = f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}'
db_connection = create_engine(db_connection_str)
connect = db_connection.connect()

### 3) 데이터 읽기

In [4]:
df = pd.read_sql("SELECT * FROM case_info", connect)
df.head()

,판례일련번호,사건명,사건번호,선고일자,선고,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용
0,64437,사해행위취소등,2003나1746,2003-07-10,선고,대전고법,4002,민사,4001,판결: 상고,파산자의 채권에 기한 사해행위취소의 소에서 사해행위 사실을 알았는지 여부를 파산관재...,파산법이 파산관재인에게 파산재단에 관한 소에 있어 원고 또는 피고가 된다고 한 것은...,"민법 제406조 제2항\n,\n\n 파산법 제7조","\n대법원 1990. 11. 13. 선고 88다카26987 판결(공1991, 57)","【원고,항소인】 파산자 주식회사 충남상호신용금고의 파산관재인 예금보험공사 (소송..."
1,64438,위증,2008도3300,2008-06-26,선고,대법원,4002,형사,4001,판결,[1] 공범인 공동피고인이 다른 공동피고인에 대한 공소사실에 관하여 증인적격이 있는...,[1] 공범인 공동피고인은 당해 소송절차에서는 피고인의 지위에 있으므로 다른 공동피...,"[1] \n \n 형사소송법 제146조, \n ...",[1] \n \n 대법원 1983. 10. 25. 선고 8...,【피 고 인】 【상 고 인】 검사【원심판결】 수원지법 2008. 4. 1. 선고 2...
2,64439,상고장각하명령에대한이의,2007무77,2008-06-02,자,대법원,4002,일반행정,4001,결정,[1] 소송구조신청에 대한 기각결정이 확정되기 전에 소장 등에 인지를 첩부하지 않았...,[1] \n \n 행정소송법 제8조 제2항에 의하여...,"[1] \n \n 민사소송 등 인지법 제1조, \n ...",[1] \n \n 대법원 1993. 1. 25.자 92마1...,【재항고인(선정당사자)】 【상 대 방】 인천광역시장【원심명령】 서울고법 2007. ...
3,64440,손해배상(기),2006다49284,2008-09-25,선고,대법원,4002,민사,4001,판결,광산에서 금광의 탐광 및 채광을 위한 굴진공사를 계속 진행할 경우 인근 주민들에게 ...,미기재,"민법 제214조, \n 제217조, \n 헌법 제35조",없음,"【원고, 피상고인 겸 상고인】 【피고, 상고인 겸 피상고인】 【원심판결】 대전고법 ..."
4,64441,임대차보증금등·지료등,"2006다62492,62508",2008-09-25,선고,대법원,4002,민사,4001,판결,"[1] 계약의 법정 또는 약정 해지사유 발생시, 당사자가 경매신청 등 계약해지를 전...",미기재,"[1] \n 민법 제105조, \n 제543조 / [2] ...",없음,"【원고(반소피고), 상고인 겸 피상고인】 【피고(반소원고), 피상고인 겸 상고인】 ..."


## 3. 모델 로드

In [5]:
model = FastText.load('./fasttext/fasttext.model')

FileNotFoundError: [Errno 2] No such file or directory: './fasttext/fasttext.model'

## 4. 임베딩 전 전처리

### 1) 문장 토큰화 전 전처리

In [8]:
df['X'] = df['판례내용'].str.replace('[^가-힣. ]', '')
df['X'].head(3)

C:\Users\Park Ji Seong\anaconda3\envs\nlp-gpu\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


0    원고항소인   파산자 주식회사 충남상호신용금고의 파산관재인 예금보험공사 소송대리인 ...
1    피 고 인 상 고 인 검사원심판결 수원지법 . . . 선고 노 판결주    문  상...
2    재항고인선정당사자 상 대 방 인천광역시장원심명령 서울고법 . . .자 누 병합 상고...
Name: X, dtype: object

### 2) 문장 토큰화
- sentence embedding을 사용할 것이므로 문장토큰화 진행

In [13]:
sentence_tokened = []

for x in tqdm_notebook(df['X']):
    sentence_tokened.append(sent_tokenize(x))

C:\Users\Park Ji Seong\anaconda3\envs\nlp-gpu\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/84735 [00:00<?, ?it/s]

## 5. 임베딩

In [36]:
embeddings = []

for sentences in tqdm_notebook(sentence_tokened):
    embedding = None
    count = 0
    for sentence in sentences:
        if embedding is None:
            embedding = model.wv.get_sentence_vector(sentence)
        else:
            embedding += model.wv.get_sentence_vector(sentence)
        count += 1
    embeddings.append(embedding / count)

C:\Users\Park Ji Seong\anaconda3\envs\nlp-gpu\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/84735 [00:00<?, ?it/s]

## 6. 임베딩 값 저장

In [39]:
with open("./data/fasttext_embedding.pickle", "wb") as fw:
    pk.dump(embeddings, fw)